In [3]:
from eBallena import eBallena
from Data import FSDD
from Baseline import generateReservoir, generateTopologyInput, assignWeightsReservoir, assignWeightsInputs
from Utils import Parallel, evalKfold, fastEval

import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [4]:
# ==============
#  LOAD DATASET
# ==============

# FSDD(time_bin=80, max_spikes=10, n_dim=26,  hop_lenght=400).preProcessAndSave(random_state=12)
fsdd = FSDD().load()

In [7]:
# ==============
#   GET METRICS
#   FUNCTION
# ==============

def get_metrics(n_neu,L,pneu_e,tau,w_in,w_re,c_ee,c_ei,c_ie,c_ii):

	# =================
	# GENERAR TOPOLOGIA
	connections_in, syn_type_in = generateTopologyInput(n_inputs=26,n_neu_reservoir=n_neu,
												  		prob_connection=0.03)
	connections_re, neuron_type_re = generateReservoir(num_neuron=n_neu, prob_exitatory=pneu_e,  L=L,
														c_ee=c_ee,c_ei=c_ei,c_ie=c_ie,c_ii=c_ii)

	syn_in = assignWeightsInputs(connections_in, syn_type_in, w_in)
	syn_re = assignWeightsReservoir(connections_re, neuron_type_re, w_re)

	# ==============
	# CONFIGURACION
	# DE SIMULACION
	config = eBallena.createConfig(n_inputs=26, n_neu=n_neu, tau=tau, max_sim_time=3_000, 
									refractory_time=5, threshold=-55)
		

	# ============================
	#        KFOLDS STAGE

	# =======================
	#     SIMULATE KFOLDS
	s = []
	t = []
	for fold in fsdd.getFolds():
		s.append( [eBallena.simulate(config,instance,syn_in,syn_re, mode='STATE') for instance in fold['d']] )
		t.append( fold['t'] )	
	# =================
	#   PARSE RESULTS

	all_acc = np.array( [evalKfold(s,t, test_idx) for test_idx in range(5)] )

	acc_folds, acc_train_folds         = all_acc.mean(axis=0).round(3)
	std_acc_folds, std_acc_train_folds = all_acc.std(axis=0).round(3)

	# ============================
	#      EVALUATION STAGE

	d_train = fsdd.train['d']
	t_train = fsdd.train['t']
	d_test  = fsdd.test['d']
	t_test  = fsdd.test['t']

	s_train = [eBallena.simulate(config,instance,syn_in,syn_re, mode='STATE') for instance in d_train]
	s_test  = [eBallena.simulate(config,instance,syn_in,syn_re, mode='STATE') for instance in d_test]

	acc_final, acc_train_final = fastEval(s_train, s_test, t_train, t_test)

	# ============================
	#      EVALUATION STAGE
	num_spikes_train = np.array(s_train).sum(axis=1).mean().round(3)
	num_spikes_test  = np.array(s_train).sum(axis=1).mean().round(3)

	return acc_folds, acc_train_folds, std_acc_folds, std_acc_train_folds, acc_final, acc_train_final, num_spikes_train, num_spikes_test


In [6]:
# ==============
#  ALL CONFS
# ==============

all_neu  = [125,512, 1000]
all_l    = [1,2,3,4,5]
all_pex  = [0.7, 0.75, 0.8, 0.85, 0.9]
all_tau  = [ 10, 20, 30, 80, 120,200]
all_w_in = [1,2,3,4,5,6]
all_w_re = [1,2,3,4,5,6,7]

all_cee  = [0.1, 0.2, 0.3]
all_cei  = [0.05, 0.1, 0.15]
all_cie  = [0.05, 0.1, 0.2, 0.3, 0.4]
all_cii  = [0.15, 0.3, 0.45]

all_configurations = []
for n_neu in all_neu:
	for L in all_l:
		for pneu_e in all_pex:
			for tau in all_tau:
				for w_in in all_w_in:
					for w_re in all_w_re:
						for c_ee in all_cee:
							for c_ei in all_cei:
								for c_ie in all_cie:
									for c_ii in all_cii:
										for _ in range(5):
											all_configurations.append( (n_neu,L,pneu_e,tau,w_in,w_re,c_ee,c_ei,c_ie,c_ii) )

In [ ]:
# ==============
#  GET RESULTS
# ==============

# n_processes = 2
results 	= []
for conf in tqdm(all_configurations[:10]):
	n_neu,L,pneu_e,tau,w_in,w_re,cee,cei,cie,cii = conf
	metrics = get_metrics(n_neu,L,pneu_e,tau,w_in,w_re,c_ee,c_ei,c_ie,c_ii)
	results.append(metrics)

	with open('results.pickle','wb') as file:
		pickle.dump( {'all_configuracions':all_configurations, 'results':results}, file )